# Daylight Saving Effect on Hypertension

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dowhy.causal_identifier import backdoor
import networkx as nx
from pgmpy.estimators import PC
from pgmpy.models import BayesianModel
import dowhy
from dowhy import CausalModel
import os
import cdt

from warnings import filterwarnings
filterwarnings('ignore')

from itertools import combinations
# cdt.SETTINGS.rpath = '/usr/lib/R/bin/Rscript' # for macOS
# cdt.SETTINGS.rpath = 'C:\Program Files\R\R-4.3.2\bin\Rscript' # for Windows
from numpy.random import normal
import pingouin as pg

np.random.seed(0)

# Motivation and Data Processing - 10% of the grade

Motivation, description of dataset and causal questions, description of assumptions, show true causal graph or a reasonable guess (10% grade)

The dataset we will use is the [healthcare dataset from Kaggle](https://www.kaggle.com/datasets/prasad22/healthcare-dataset). It contains 10,000 rows where each row represents a patients healthcare record. To guarantee patients' privacy and comply with healthcare regulations, no real patient information is used. Instead the data is generated based on actual healthcare records and it is made to consist of a similar structure and contain similar attributes. It seems to be oriented at healthcare records from the United States based on the generated data, although no clarity is provided regarding their sources on their Kaggle page.

The dataset includes the medical condition for which the patients are admitted to the hospital. We will focus on the hypertension condition which is an sustained increased blood pressure. Untreated hypertension can, among other problems, cause heart- and vascular diseases and can lead to life threatening complications such as a heart attack or heart failure [**TODO: INSERT PAPER**]. 

Hypertension has a variety of causes including a lack of sleep [**TODO: INSERT PAPER**]. This forms the basis for our causal question in which we want to find if the daylight saving time (DST) in spring (the month march) causes more patients to be admitted for hypertension. We will investigate the effect that DST has on hypertension.

[**TODO: Note**: insert paper that inspired us]

[**TODO: observational data? but fake**]

In [ ]:
file_path = '../data/healthcare/healthcare_dataset.csv'
df = pd.read_csv(file_path)

In [ ]:
# file_path = 'data/healthcare/healthcare_dataset.csv'
# df = pd.read_csv(file_path)

We start with an insight in the unique medical conditions in the 'Medical Condition' column and create our outcome variable 'Hypertension'. It is a one-hot encoded column containing a 1 if a patient is admitted for hypertension and 0 otherwise.

In [ ]:
print(df['Medical Condition'].value_counts())

df = pd.concat([df, pd.get_dummies(df['Medical Condition'], dtype='int')], axis=1)
df = df.drop(columns='Medical Condition')
df = df.drop(columns=['Asthma', 'Cancer', 'Arthritis', 'Obesity', 'Diabetes'])
df.head()

The dataset contains multiple entries with the same patient name. We only keep first entry of the same person since we do not want to include subsequent related hosipital admissions since they could span over a longer period than our specic timepoint of the DST.

[**TODO: same names != same person. should have checked on age & Name atleast** maybe our reasoning could be that we had an oversight and did not expect the pyton faker module to generate healthcare records for different people with the same name]

In [ ]:
filtered_df = df[~df['Name'].duplicated(keep='first')]
print(f'{df.shape[0] - filtered_df.shape[0]} entries are removed')

Let's see how many unique values each variable has.

In [ ]:
for col_name in filtered_df.columns:
    print(f'{col_name}:{filtered_df[col_name].nunique()}')

We decide that the following variables are irrelevant for our analysis or have arbitrary data: ```['Doctor', 'Hospital', 'Room Number', 'Discharge Date']```. We can also drop ```'Name'``` since we already filtered by the first visit of unique individuals.

In [ ]:
filtered_df = filtered_df.drop(columns=['Doctor', 'Hospital', 'Room Number', 'Discharge Date', 'Name'])

Let's see the unique values of the columns with less than 10 unique values. We print the column names containing continuous data.

In [ ]:
categoricals = []

for col_name in filtered_df.columns:
    if filtered_df[col_name].nunique() <= 10:
        print(f'{col_name}:{filtered_df[col_name].unique()}')
        categoricals.append(col_name)
    else:
        print(f'{col_name} is a continuous variable.')

Let's examine the seasonal patterns in hospital admissions and compare it with the trends observed among patients diagnosed with Hypertension. This allows us to determine the range of the period in which we want to look for the effect of the DST time change.

In [ ]:
# Note: y-axis does not start at 0
filtered_df['Date of Admission'] = pd.to_datetime(filtered_df['Date of Admission'])
plt.figure(figsize=(12, 5))

# admitted patients without hypertension
plt.subplot(1, 2, 1)
doy_frequency = filtered_df['Date of Admission'][filtered_df.Hypertension == 0].apply(lambda x: x.month).value_counts().sort_index(ascending=True)
plt.step(doy_frequency.index, doy_frequency.values, where='post', color='blue', alpha=0.7)
plt.xticks(range(1, 13))
plt.title('Monthly Hospital Admissions for Patients without Hypertension')
plt.xlabel('Month')
plt.ylabel('Number of Admitted Patients')
plt.grid(axis='y', linestyle='--', alpha=0.7)

# admitted patients with hypertension
plt.subplot(1, 2, 2)
doy_frequency = filtered_df['Date of Admission'][filtered_df.Hypertension == 1].apply(lambda x: x.month).value_counts().sort_index(ascending=True)
plt.step(doy_frequency.index, doy_frequency.values, where='post', color='blue', alpha=0.7)
plt.xticks(range(1, 13))
plt.title('Monthly Hospital Admissions for Patients with Hypertension')
plt.xlabel('Month')
plt.ylabel('Number of Admitted Patients')
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()


In the graph depicting the number of patients admitted to the hospital for Hypertension, the two months following March exhibit higher admission counts compared to the preceding months. However, when examining the admissions for other medical conditions, the month of February similarly shows significantly fewer admissions than march.

To isolate the effect of DST on hypertension, we choose to investigate a 20-day period after DST in spring. The limits the influence of the montly fluctuations in the datset and is in line with similar papers which examine the effect of sleep shortage on hypertension [**TODO: insert paper annelouse found**]. 



To create a variable which encodes the DST change in spring, we aim to introduce a binary column: assigning a value of 1 if the admission date falls within a 10-day window surrounding the annual DST change in March (a period during which individuals typically adjust their schedules due to a one-hour reduction in sleep), and 0 otherwise. Our dataset spans the years 2018-2023, and we construct a dictionary pairing each year with the specific date of the daylight saving time transition.

In [ ]:
daylight_saving_dates = {
    2018: '2018-03-25',
    2019: '2019-03-31',
    2020: '2020-03-29',
    2021: '2021-03-28',
    2022: '2022-03-27',
    2023: '2023-03-26'}

In [ ]:
daylight_saving_dates = {year: pd.to_datetime(date) for year, date in daylight_saving_dates.items()}

filtered_df['Date of Admission'] = pd.to_datetime(filtered_df['Date of Admission'])

filtered_df['daylight_saving_march'] = filtered_df['Date of Admission'].apply(lambda date: 1 
                                                                        if daylight_saving_dates[date.year] <= date <= daylight_saving_dates[date.year] + pd.DateOffset(days=20)
                                                                        else 0)

filtered_df['daylight_saving_before_march'] = filtered_df['Date of Admission'].apply(lambda date: 1 
                                                                        if daylight_saving_dates[date.year] >= date >= daylight_saving_dates[date.year] - pd.DateOffset(days=20)
                                                                        else 0)

Let's see how balanced is our data.

In [ ]:
for col_name in filtered_df.columns:
    if filtered_df[col_name].nunique() <= 10:
        print(filtered_df[col_name].value_counts())
        print()

[**TODO** conclusion of databalance]

[**TODO: maybe in first initial dataset description sesction, Describe the assumptions of your dataset (causal sufficiency, no cycles in the causal graph, positivity, etc).**]

# Exploratory Data Analysis - 15% of the grade

Testing correlations/conditional independences (15% grade, follow Tutorial 1&2)

## Our estimate of the causal graph
We start with reasoning about the relevant variables and their connections to create an estimation for the causal graph afer which we will perform independence tests and correct the graph if necessary.

In [ ]:
for col_name in filtered_df.columns:
    print(col_name)

In [ ]:
graph_variables = ['Age', 'Gender', 'Insurance Provider', 'Billing Amount', 
                   'Admission Type', 'Medication', 'Test Results', 'Hypertension', 'daylight_saving_march']

G = nx.DiGraph()
G.add_nodes_from(graph_variables)

edges = [
    ('Insurance Provider', 'Billing Amount'),       # decides the billing amount
    ('Age', 'Billing Amount'),                      # older people require more care for same treatment
    ('Age', 'Insurance Provider'),                  # marketing or preference by age
    ('Age', 'Hypertension'),                        # blood pressure rises and likelihood of hypertension 
    ('Age', 'Medication'),                          # older people might get different medication
    ('Gender', 'Hypertension'),                     # hormonal reasons maybe influnece the likelihood of hypertension
    ('Gender', 'Medication'),                       # not all medicine apply to men or women
    ('Hypertension', 'Test Results'),               # the medical condition influences the outcome of the tests           
    ('Test Results', 'Medication'),                 # medication is prescribed based on the outcome of the tests
    ('Medication', 'Billing Amount'),               # billing amount is dependent on the prescribed medication
    ('daylight_saving_march', 'Hypertension'),      # hypothesis: due to lack of sleep
    ('Admission Type', 'Billing Amount'),           # emerGgency requires more money
    ('daylight_saving_march', 'Admission Type'),    # ?? the period after spring DST effects the number of emergencies etc.
    ('Age', 'daylight_saving_march'),               # ?? confounder: see explanation below
]


G.add_edges_from(edges)

pos = nx.kamada_kawai_layout(G)
node_size = [800 if node in graph_variables else 200 for node in G.nodes]
node_color = ['skyblue' if node in graph_variables else 'lightgray' for node in G.nodes]

nx.draw(G, pos, with_labels=True, font_size=6, node_size=node_size, node_color=node_color, font_color='black', font_weight='bold', arrowsize=10)

plt.tight_layout()
plt.show()

In [ ]:
# Check for cycles
list(nx.simple_cycles(G))

## Independence tests
Let's identify the correlation between the variables in the dataset.

In [ ]:
from itertools import permutations
import pingouin as pg

def test_all(df, vars):
    # Marginal
    for var1, var2 in permutations(vars, 2):
        p_val = pg.partial_corr(data=df, x=var1, y=var2, covar=[], method='pearson')['p-val'].item()
        print('{} and {}: p-value is {}'.format(var1, var2, p_val))

    # Conditional
    for var1, var2, cond in permutations(vars, 3):
        p_val = pg.partial_corr(data=df, x=var1, y=var2, covar=[cond], method='pearson')['p-val'].item()
        print('{} and {} given {}: p-value is {}'.format(var1, var2, cond, p_val))

In [ ]:
def encode_categoricals(df):
    from sklearn.preprocessing import LabelEncoder

    label_encoder = LabelEncoder()

    for column in df.columns:
        if df[column].dtype == 'object':
            df[column] = label_encoder.fit_transform(df[column])
            
    return df

In order to have the partial_corr independence test work we need to have numerical columns encoded into categorical data

In [ ]:
df_encoded = encode_categoricals(filtered_df)
df_encoded

In [ ]:
test_all(df_encoded, graph_variables)

## Interpretation on results

We expect to get significant p-values for all tests where the graph implies dependece between the variables. We list below some significant conclusions on the p-value.

### Independences

From the independence tests we can notice the low p-values which are below 0.1 the following imply independence:

- gender and insurance provider on their own, but also conditioning on any variable, also accoring to the graph

Gender and Insurance Provider: p-value is 0.05167704764915217

- insurance provider and billing amount, however in the graph we decided there should be some dependence.

Insurance Provider and Billing Amount: p-value is 0.0820549226831016

- Insurance provider is also independent from gender, also according to our graph

Insurance Provider and Gender: p-value is 0.05167704764915217

- medication and daylight saving march, not according with our graph because there is a direct path between the two variables: daylight_saving_march --> Hypertension --> Test Results --> Medication

### Dependences

The high p-values show us the dependence between the variables:
- also according to our graph, there is a high dependence between insurance provider and age

Insurance Provider and Age: p-value is 0.930665278923154


- Insurance Provider and medication are dependent because they are cofounded by Age, also according to our graph

Insurance Provider and Medication: p-value is 0.6086810195977133

- Hypertension is highly dependent on Insurance Provider, also a conclusion aligning with our graph

Insurance Provider and Hypertension: p-value is 0.9939665403739955

- Tests Result is highly correlated with daylight saving, also according to our graph. Test Result seems to be dependent on all the variables, this is also reflected in the graph we came up with which seems great.

Test Results and daylight_saving_march: p-value is 0.6844684976503201

- Billing amount is dependent on gender, not accordyng to our graph.

Billing Amount and Gender: p-value is 0.6399470393661836

## Modifications of the graph after the independence tests

Given the results form the independence tests, we will make the following modifications in our graph:

- Insurance provider will not directly affect billing amount
- insurance provider directly influences hypertension, and age is a confounder for the two
- add an arrow between billing amount and gender
- we eliminate medication and test results as nodes because they might be time dependent (is the medication taken after the test results)

In [ ]:
graph_variables = ['Age', 'Gender', 'Insurance Provider', 'Billing Amount', 
                   'Admission Type', 'Hypertension', 'daylight_saving_march'] #, 'Arthritis', 'Asthma', 'Cancer', 'Diabetes', 'Obesity',]

G = nx.DiGraph()
G.add_nodes_from(graph_variables)

edges = [
    ('Age', 'Billing Amount'),                      # older people require more care for same treatment
    ('Age', 'Insurance Provider'),                  # marketing or preference by age
    ('Age', 'Hypertension'),                        # blood pressure rises and likelihood of hypertension 
    ('Gender', 'Hypertension'),                     # hormonal reasons maybe influnece the likelihood of hypertension
    ('daylight_saving_march', 'Hypertension'),      # hypothesis: due to lack of sleep
    ('Admission Type', 'Billing Amount'),           # emerGgency requires more money
    ('daylight_saving_march', 'Admission Type'),    # ?? the period after spring DST effects the number of emergencies etc.
    ('Age', 'daylight_saving_march'),               # ?? confounder: see explanation below
    ('Insurance Provider', 'Hypertension'),        # independence test modification
    ('Billing Amount', 'Gender')
]


G.add_edges_from(edges)

pos = nx.kamada_kawai_layout(G)
node_size = [800 if node in graph_variables else 200 for node in G.nodes]
node_color = ['skyblue' if node in graph_variables else 'lightgray' for node in G.nodes]

nx.draw(G, pos, with_labels=True, font_size=6, node_size=node_size, node_color=node_color, font_color='black', font_weight='bold', arrowsize=10)
nx.write_gml(G, "causal_graph.gml")

plt.tight_layout()
plt.show()

In [ ]:
# Check for cycles
list(nx.simple_cycles(G))

## Exploring the hypertension admission frequencies surrounding DST spring

Before continuing with the causal analysis, lets consider the statstics for patients with hypertension before and after the DST time change.

In [ ]:
# AFTER daylight saving time (from last sunday of march)
n_hyp = filtered_df[filtered_df.Hypertension == 1].shape[0]
n_hyp_after = filtered_df[filtered_df.Hypertension == 1][filtered_df['daylight_saving_march'] == 1].shape[0]
perc_hyp_after = (n_hyp_after / n_hyp)*100

n_after_march = filtered_df[filtered_df['daylight_saving_march'] == 1].shape[0]
perc_hyp_after_total = (n_hyp_after / n_after_march) * 100
print(f'Of the {round(n_hyp, 2)} admissions for hypertension, {round(perc_hyp_after, 2)} percent was in the 20-days AFTER daylight saving in march')
print(f'This formed {round(perc_hyp_after_total, 2)} percent of the total of {round(n_after_march, 2)} admissions for any condition in the same time period')

In [ ]:
# BEFORE daylight saving march
n_hyp_before = filtered_df[filtered_df.Hypertension == 1][filtered_df['daylight_saving_before_march'] == 1].shape[0]
perc_hyp_before = (n_hyp_before / n_hyp)*100

n_before_march = filtered_df[filtered_df['daylight_saving_before_march'] == 1].shape[0]
perc_hyp_before_total = (n_hyp_before / n_before_march) * 100
print(f'Of the {round(n_hyp, 2)} admissions for hypertension {round(perc_hyp_before, 2)} percent was in the 20-days BEFORE daylight saving in march')
print(f'This formed {round(perc_hyp_before_total, 2)} percent of the total of {round(n_before_march, 2)} admissions for any condition in the same time period')

We notice that 5.69% of Hypertension cases happen within 20 days after the daylight saving in March, while 5.06% of the hypertension cases happen before the daylight saving time. 

From the plot with the patients admitted per month showed that there is a deviation from month to month in the number of admissions. Therefore we also need to look at the proportion of hypertension cases compared to the total hospital admissions.

We see that the proportion of Hypertension cases among all hospital admissions is also greater in the 20 days following the DST time change compared to 20 days preceding the time change.

# Identify estimands for backdoor, frontdoor criterion and IVs - 20% of the grade

## Finding all paths between ```daylight_saving_march``` and ```Hypertension```

In [ ]:
all_paths = list(nx.all_simple_paths(G.to_undirected(), source='daylight_saving_march', target='Hypertension'))
print('Number of paths found:', len(all_paths))

if len(all_paths) > 20:
    all_paths = all_paths[:20]
    
n_cols = 3
n_rows = (len(all_paths) - 1) // n_cols + 1
fig, axs = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(6 * n_cols, 6 * n_rows))

for i, path in enumerate(all_paths):
    col = i % n_cols
    row = i // n_cols
    edges_on_path = [set([source, target]) for source, target in zip(path[:-1], path[1:])]
    edge_color = ['r' if set(edge) in edges_on_path else 'black' for edge in G.edges()]  
    nx.draw_shell(G, with_labels=True, node_color=['w' if i < len(graph_variables) - 2 else 'r' for i in range(len(graph_variables))], edgecolors='black', ax=axs[row, col], edge_color=edge_color)
    axs[row, col].set_title(path[1:-1])
    
# hide empty plots
for i in range(len(all_paths), n_rows * n_cols):
    axs.flatten()[i].axis('off')

Daylight saving martch and hypertension are not d-separated because they have a direct connection.

### Backdoor Criterion

The backdoor criterion allows us to identify the variables on which we need to condition to calculate our causal estimates. We identify these variables by looking at 'backdoor' paths from ```daylight_saving_march``` and ```Hypertension```. We can apply the backdoor criterion here because there are arrows that go into our treatment, ```daylight_saving_march```, and many confounder on the way for which we need to adjust.


In [ ]:
from dowhy import CausalModel

# a utility function to parse the .gml file to string
def gml_to_string(file):
    gml_str = ''
    with open(file, 'r') as file:
        for line in file:
            gml_str += line.rstrip()
    return gml_str

gml_graph = gml_to_string('causal_graph.gml')
# With GML string
model=CausalModel(
    data = df_encoded,
    treatment='daylight_saving_march', 
    outcome='Hypertension',
    graph=gml_graph
)
model.view_model()

In [ ]:
# importing the backdoor class from DoWhy
from dowhy.causal_identifier import backdoor

df_encoded.head()

# creating a copy of our graph G that is undirected
H = G.to_undirected()
# all_possible_paths = list(nx.all_simple_paths(H, 'daylight_saving_march', 'Hypertension'))
bd = backdoor.Backdoor(G, 'daylight_saving_march','Hypertension')
backdoor_paths = [path for path in all_paths if bd.is_backdoor(path)]

#### Visualization of backdoor paths

In [ ]:
# we apply the is_backdoor function to each path to check if a path is a backdoor path

print('Number of backdoor paths found:', len(backdoor_paths))

if len(backdoor_paths) > 20:
    backdoor_paths = backdoor_paths[:20]
    
n_cols = 3
n_rows = (len(backdoor_paths) - 1) // n_cols + 1
fig, axs = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(6 * n_cols, 6 * n_rows))

for i, path in enumerate(backdoor_paths):
    col = i % n_cols
    row = i // n_cols
    edges_on_path = [set([source, target]) for source, target in zip(path[:-1], path[1:])]
    edge_color = ['r' if set(edge) in edges_on_path else 'black' for edge in G.edges()]  
    nx.draw_shell(G, with_labels=True, node_color=['w' if i < len(graph_variables) - 2 else 'r' for i in range(len(graph_variables))], edgecolors='black', ax=axs[col], edge_color=edge_color)
    axs[col].set_title(path[1:-1])
    
# hide empty plots
for i in range(len(backdoor_paths), n_rows * n_cols):
    axs.flatten()[i].axis('off')

We find three backdoor paths between our treatment and the outcome. Let's analyse what we need to adjust for.

In [ ]:
adjustment_variables = pd.DataFrame(columns=['path', 'colliders', 'non_colliders'])

In [ ]:
for path in backdoor_paths:
    colliders = np.array([])
    non_colliders = []
    path_len = len(path)

    # we loop through adjacent variables on the path, ignoring the source and target variables as potential colliders
    for node0, node1, node2 in zip(path[0:path_len-2], path[1:path_len-1], path[2:]):
        # if there is an arrow pointing into node1 from both sides on the path, it is a collider
        if G.has_edge(node0, node1) and G.has_edge(node2, node1):
            colliders = np.append(colliders, list(nx.descendants(G,node1)) + [node1]) # so we add it (and all its descendants) to the list
    # we flatten the list of list
    colliders = colliders.flatten()

    # any node on the path (excluding the source and target) that is not a collider is a non-collider
    non_colliders = [x for x in path[1:-1] if x not in colliders]

    # finally, we add the information to our dataframe, with the path, colliders, and non-colliders
    adjustment_variables.loc[len(adjustment_variables.index)] = [path, colliders, non_colliders] 

In [ ]:
adjustment_variables

There are no colliders on any of the backdoor paths. We need to condition on at least one common non-collider from the three paths found, which are not in the collider set. Therefore, we need to condition on ```Age``` to block all backdoor paths from our treatment ```daylight_saving_march``` to our outcome ```Hypertension```.

### Frontdoor Criterion

We want to also apply the frontdoor criterion. We want to find the adjustment set which satisfies the frontdoor criterion. First of all we need to get all directed paths from ```daylight_saving_march``` and ```Hypertension```.

In [ ]:
our_causal_graph = model._graph
our_causal_graph.get_all_directed_paths(["daylight_saving_march"], ["Hypertension"])

Because there is a direct edge between our treatment and our outcome, we cannot apply the frontdoor criterion. The adjustment set able to satiisfy this criterion needs to intercent all directed patch from ```daylight_saving_march``` and ```Hypertension``` and because of the direct edge between the two variables, such an adjustment set does not exist.

### Instrumental Variables

We believe there is an unobserved confounder, ```sleep_quality```, between ```daylight_saving_march``` and ```Hypertension``` because one less hour of sleep could directly affect the hypertension risks and the date of admission to the hospital.

The only measure that we have which could influece the date of admission (implicitly ```daylight_saving_march```) is ```Age```. Depending on age, people have different routines during the year, moments where stress is higher, or they simply have a higher risk of getting a heart attack.

However, ```Age``` cannot be part of the adjustment set for the Instrumental Variables criterion because it has a direct effect on ```Hypertension```. Therefore, there are no variables found to adjust for the iv.

### DoWhy Check

In [ ]:
identified_estimand = model.identify_effect()
print(identified_estimand)

identifier = model.identifier
identifier.identify_backdoor(model._graph, model._treatment, model._outcome)

Indeed, we need to adjust for ```Age``` to block all the backdoor paths. There is no adjustment set found for the frontdoor criterion or the instrumental variables criterion.

# Estimate the causal effects

Given our reasoning from the previous section, we know that we have to condition on ```Age``` to get the true effect of ```daylight_saving_march``` on ```Hypertension```. We apply different estimators from the DoWhy package.

#### Linear Estimator

In [ ]:
causal_estimate = model.estimate_effect(identified_estimand,
        method_name="backdoor.linear_regression")
print(causal_estimate)
print("Causal Estimate is " + str(causal_estimate.value))

In [ ]:
# Textual Interpreter
causal_estimate.interpret(method_name="textual_effect_interpreter")

With a linear estimator we have a causal effect of -0.02849093. We can see that maybe a linear estimator is not the best to apply. We might think that the true nature of our effect and the assumptions made by the estimator do not match entirely. We would expect for a positive causal estimate effect of the ```daylight_saving_march``` and ```Hypertension```. We therefore apply one more estimate

#### Propensity Score Stratification

In [ ]:
causal_estimate_strat = model.estimate_effect(identified_estimand,
                                              method_name="backdoor.propensity_score_stratification",
                                              target_units="att")
print(causal_estimate_strat)
print("Causal Estimate is " + str(causal_estimate_strat.value))

In [ ]:
# Textual Interpreter
interpretation = causal_estimate_strat.interpret(method_name="textual_effect_interpreter")

#### Propensity Score Matching

In [ ]:
causal_estimate_match = model.estimate_effect(identified_estimand,
                                              method_name="backdoor.propensity_score_matching",
                                              target_units="atc")
print(causal_estimate_match)
print("Causal Estimate is " + str(causal_estimate_match.value))

In [ ]:
# Textual Interpreter
interpretation = causal_estimate_match.interpret(method_name="textual_effect_interpreter")

We notice that we finally get a positive effect of the ```daylight_saving_march``` on ```Hypertension``` when using a propensity score matching. Weirdly enough, when the data is startified and using a propensity score we get a causal effect of -0.028. When match units in data instead we obtain a positive causal effect 0.00461.

# Finding the Causal Graph with Causal Discovery

#### Obtain skeleton of graph

In [ ]:
nodes =  graph_variables
skeleton = nx.Graph()
skeleton.add_nodes_from(nodes)
skeleton.add_edges_from(combinations(nodes, 2))

nx.draw(skeleton, with_labels=True, node_color='w', edgecolors ='black')

#### Apply independence tests to get the undirected graph

In [ ]:
# initialize graph lasso
import cdt

glasso = cdt.independence.graph.Glasso()

# apply graph lasso to data
df = df_encoded[graph_variables]
skeleton = glasso.predict(df) # visualize network
fig = plt.figure(figsize=(15,10))
nx.draw_networkx(skeleton, font_size=18, font_color='r')

Apply PC

In [ ]:
import cdt
pc = cdt.causality.graph.PC(CItest="gaussian", alpha=0.01)

pc_graph = pc.create_graph_from_data(df)
colors = ['red'  if (y,x) in pc_graph.edges() else 'black' for (x,y) in pc_graph.edges()]
nx.draw(pc_graph, with_labels=True, node_size=100, node_color='w', edgecolors ='black', edge_color=colors)

In [ ]:
pc_graph.edges()

In [ ]:
new_skeleton = cdt.utils.graph.remove_indirect_links(skeleton, alg='aracne')
model = cdt.causality.graph.GES()
output_graph = model.predict(df, new_skeleton)
print(nx.adjacency_matrix(output_graph).todense())

nx.draw(output_graph, with_labels=True, node_size=100, node_color='w', edgecolors ='black', edge_color=colors)

# Using synthetic data based on our prior

As we could not find many causal relations and aim to show our implementation of our algorithms further, we simulate data that resembles our real data. In this case we use less edges. The simulation of synthetic data is a common practise. Real data might be hard to obtain, or due to confidentiality issues it might be hard to get acces to. Privacy especially important in the case of medical data. 

It is hypothesised that the alteration in time at DST notably affects our sleep patterns. Studies indicate that following the spring time adjustment, more people encounter reduced sleep duration and an increase in disturbances in their sleep cycle. Such disruptions of sleep can lead to elevated heart rate, increased blood pressure, and a heightened risk of arrhythmias.

For illustration we add the mediator "Sleep Quality" between 'daylight_saving_march' and 'hypertension. 

$$
\begin{aligned}
AGE: X_1 &\leftarrow \epsilon_1 & \epsilon_1 &\sim \mathcal N(0,1)\\ 

MEDICATION: X_2 &\leftarrow X_1 + X_3 + X_4 + \epsilon_2 & \epsilon_2 &\sim \mathcal N(0,0.7)\\
TEST RESULTS: X_3 &\leftarrow X_5 + \epsilon_3 & \epsilon_3 &\sim \mathcal N(0,1)\\
GENDER: X_4 &\leftarrow \epsilon_4 & \epsilon_4 &\sim \mathcal N(0,1)\\
HYPERTENSION: X_5 &\leftarrow X_4 + X_1 + X_6 + \epsilon_5 & \epsilon_5 &\sim \mathcal N(0,1)\\
SLEEP QUALITY: X_6 &\leftarrow 2 X_7 + \epsilon_6 & \epsilon_6 &\sim \mathcal N(0,1.5)\\
DAYLIGHT SAVING MARCH: X_7 &\leftarrow 3 X_1 + \epsilon_7 & \epsilon_7 &\sim \mathcal N(0,1)\\
\end{aligned}
$$

In [ ]:
n_samples=10000

eps1 = normal(0, 1, n_samples)
eps2 = normal(0, 0.7, n_samples)
eps3 = normal(0, 1, n_samples)
eps4 = normal(0, 1, n_samples)
eps5 = normal(0, 1, n_samples)
eps6 = normal(0, 1.5, n_samples)
eps7 = normal(0, 1, n_samples)

X1 = eps1
X4 = eps4
X7= 3 * X1 + eps7
X6 = 2* X7 + eps6
X5 = X4 + X1 + X6 + eps5
X3 = X5 + eps3
X2 = X1 + X3 + X4 + eps2

# we can plot scatter plots between any two variables 
plt.scatter(X1, X2)

In [ ]:
plt.scatter(X4, X7)

In [ ]:
from GENCDA.data_generation.relatedDataframe import RelatedDataframe
from GENCDA.data_generation.randomDataframe import randomDataframe

num_samples = 1500
# Define a DAG
edges = [
    # ('Insurance Provider', 'Billing Amount'),       # decides the billing amount
    # ('Age', 'Billing Amount'),                      # older people require more care for same treatment
    # ('Age', 'Insurance Provider'),                  # marketing or preference by age
    ('Age', 'Hypertension'),                        # blood pressure rises and likelihood of hypertension 
    ('Age', 'Medication'),                          # older people might get different medication
    ('Gender', 'Hypertension'),                     # hormonal reasons maybe influnece the likelihood of hypertension
    ('Gender', 'Medication'),                       # not all medicine apply to men or women         
    ('Hypertension', 'Test Results'),           
    ('Test Results', 'Medication'),
    # ('Medication', 'Billing Amount'), 
    ('daylight_saving_march', 'Sleep Quality'),       # reduced sleep
    ('Sleep Quality', 'Hypertension'),      # hypothesis: due to lack of sleep
    # ('Admission Type', 'Billing Amount'),           # emerGgency requires more money
    # ('daylight_saving_march', 'Admission Type'),    # The period after spring DST effects the number of emergencies etc.
    ('Age', 'daylight_saving_march'),              
]


dag = nx.to_networkx_graph(edges, create_using=nx.DiGraph)

random_data = randomDataframe(df)
random_data.columns = df.columns

print(random_data.head())


In [ ]:
# edges = [
#     # ('Insurance Provider', 'Billing Amount'),       # decides the billing amount
#     # ('Age', 'Billing Amount'),                      # older people require more care for same treatment
#     # ('Age', 'Insurance Provider'),                  # marketing or preference by age
#     ('Age', 'Hypertension'),                        # blood pressure rises and likelihood of hypertension 
#     ('Age', 'Medication'),                          # older people might get different medication
#     ('Gender', 'Hypertension'),                     # hormonal reasons maybe influnece the likelihood of hypertension
#     ('Gender', 'Medication'),                       # not all medicine apply to men or women         
#     ('Hypertension', 'Test Results'),           
#     ('Test Results', 'Medication'),
#     # ('Medication', 'Billing Amount'), 
#     ('daylight_saving_march', 'Sleep Quality'),       # reduced sleep
#     ('Sleep Quality', 'Hypertension'),      # hypothesis: due to lack of sleep
#     # ('Admission Type', 'Billing Amount'),           # emerGgency requires more money
#     # ('daylight_saving_march', 'Admission Type'),    # The period after spring DST effects the number of emergencies etc.
#     ('Age', 'daylight_saving_march'),              
# ]


# dag = nx.to_networkx_graph(edges, create_using=nx.DiGraph)

# nx.draw(dag, with_labels=True, node_size=100, node_color='w', edgecolors ='black', edge_color=colors)

In [ ]:
new_synthetic_df = pd.DataFrame({'Age': X1,'Medication': X2,'Test Results': X3,'Gender': X4,'Hypertension': X5, 'Sleep Quality': X6,'Daylight saving march': X7})
new_synthetic_df.head()

In [ ]:
new_synthetic_df = pd.DataFrame({'Age': X1,'Medication': X2,'Test Results': X3,'Gender': X4,'Hypertension': X5, 'Sleep Quality': X6,'Daylight saving march': X7})
# testing the GES on the newly generated syntehtic data
new_skeleton = cdt.utils.graph.remove_indirect_links(skeleton, alg='aracne')
model = cdt.causality.graph.GES()
output_graph = model.predict(new_synthetic_df, new_skeleton)

print(nx.adjacency_matrix(output_graph).todense())

nx.draw(output_graph, with_labels=True, node_size=100, node_color='w', edgecolors ='black')

In [ ]:
# testing the PC on the newly generated syntehtic data
pc = cdt.causality.graph.PC(CItest="gaussian", alpha=0.01)

pc_graph = pc.create_graph_from_data(new_synthetic_df)
colors = ['red'  if (y,x) in pc_graph.edges() else 'black' for (x,y) in pc_graph.edges()]
nx.draw(pc_graph, with_labels=True, node_size=100, node_color='w', edgecolors ='black')

In [ ]:
# LINGAM
import networkx as nx
from cdt.causality.graph import LiNGAM

obj = LiNGAM()
output = obj.predict(new_synthetic_df)
colors = ['red'  if (y,x) in pc_graph.edges() else 'black' for (x,y) in pc_graph.edges()]
nx.draw(output, with_labels=True, node_size=100, node_color='w', edgecolors ='black', edge_color=colors)


In [ ]:
for edge in output.edges():
    print(edge)

### GIES

In [ ]:
# GIES
from cdt.causality.graph import GIES

obj = GIES()
#The predict() method works without a graph, or with a
#directed or undirected graph provided as an input

# output = obj.predict(new_synthetic_df)    #No graph provided as an argument
# output = obj.predict(new_synthetic_df, nx.Graph(graph))  #With an undirected graph

output = obj.predict(new_synthetic_df, dag)  #With a directed graph

#To view the graph created, run the below commands:
nx.draw_networkx(output, font_size=8)
plt.show()

# Conclusion

Discussion on the assumptions and results